In [45]:
import pandas as pd
from torchtext.data import Field, BucketIterator
import torch
import spacy
import en_core_web_sm
svoc = en_core_web_sm.load()
import numpy as np
from torchtext.data import TabularDataset

In [4]:
datao = pd.read_pickle("~/OneDrive/kph/processed.pkl")

In [41]:
# filter out less interesting keywords
datao['ext perc'].value_counts()

3     5403
4     4737
2     3903
5     2703
1     1883
6     1032
0      746
7      305
8      120
9       46
10      35
11      17
13       8
12       6
15       5
14       3
16       3
17       2
22       2
18       1
Name: ext perc, dtype: int64

In [27]:
# choose everything larger than 3 as the training and testing data set
datatrain = datao[datao['ext perc']>=3]
datatest = datao[datao['ext perc']<3]

In [39]:
# prepare the train, test, validate data set
def prepare_csv(df_train, df_test,VAL_RATIO, seed=250):
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv(
        "~/OneDrive/kph/ttd/train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv(
        "~/OneDrive/kph/ttd/val.csv", index=False)
    df_test.to_csv("~/OneDrive/kph/ttd/test.csv", index=False)

In [40]:
prepare_csv(datatrain.loc[:,['SRC','TRG']], datatest.loc[:,['SRC','TRG']],0.2, seed=250)

In [17]:
tokenizertrg = lambda x: x.split()
def tokenizersrc(text): # create a tokenizer function
    return [tok.text for tok in svoc.tokenizer(text)]

In [43]:
SRC = Field(sequential=True, tokenize=tokenizersrc)
TRG = Field(sequential=True, tokenize=tokenizertrg, use_vocab=False)

In [46]:
data_fields = [('SRC', SRC), ('TRG', TRG)]

In [56]:
train, val, test = TabularDataset.splits(
    path='~/OneDrive/kph/ttd/', train='train.csv',
    validation='val.csv', test='test.csv', format='csv',
    fields=data_fields,skip_header=True)


In [61]:
# first version with fixed embeddings
from torchtext import vocab

In [71]:
embvec = vocab.GloVe(name='840B', dim=300,cache='/home/pding/Documents/glove/')

100%|█████████▉| 2196016/2196017 [03:35<00:00, 10178.84it/s]


In [ ]:
SRC.build_vocab(train, vectors=embvec)

In [ ]:
#TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
#embedding = nn.Embedding(n_embed, embed_dim).from_pretrained(TEXT.vocab.vectors)

In [ ]:
from torchtext.data import Iterator, BucketIterator

In [ ]:

train_iter = data.BucketIterator(
    dataset=mt_train, batch_size=32,
    sort_key=lambda x: len(x.comment_text))